Rajout des infos venant de l'API Spotify pour l'artiste principal:
- nb de followers
- genres
- marchés

In [2]:
import requests
import os
import pandas as pd

TOKEN_URL = 'https://accounts.spotify.com/api/token'
BASE_URL = 'https://api.spotify.com/v1/'

# 1. Retrieve access token and call Spotify API

In [3]:
## Envoyer une requête POST avec nos credentials 
CLIENT_ID=os.environ.get('CLIENT_ID')
CLIENT_SECRET=os.environ.get('CLIENT_SECRET')

data = {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
}

headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

auth_response = requests.post(TOKEN_URL, headers=headers, data=data)
print(auth_response.json()) # returns a dictionary
print(auth_response.json()['access_token'])

{'access_token': 'BQCE3ElwpgYu-yEjKN7Gc2cYNh3ybU5Tu8GwZebck0fWNDSScMv9ENOkhswuPwJ3iRa2wEJowV0UNk0sJQkFZr2lwqnfJr2aYJawcxg7sWVeP-lKPCA3i5TCrHaP1ospW7YpYbDMOMk', 'token_type': 'Bearer', 'expires_in': 3600}
BQCE3ElwpgYu-yEjKN7Gc2cYNh3ybU5Tu8GwZebck0fWNDSScMv9ENOkhswuPwJ3iRa2wEJowV0UNk0sJQkFZr2lwqnfJr2aYJawcxg7sWVeP-lKPCA3i5TCrHaP1ospW7YpYbDMOMk


In [4]:
# RECUPERER DATA EN NOUS AUTHENTIFIANT AVEC L'ACCESS TOKEN ex. info d'un artiste spécifique
## Définir le headers avec l'access_token
## Penser à utiliser f-string ou .format() pour introduire l'access_token défini ci-dessus
access_token = auth_response.json().get('access_token') # access token à mettre en variable d'env pour pas qu'un user public réutilise
print(access_token)

BQCE3ElwpgYu-yEjKN7Gc2cYNh3ybU5Tu8GwZebck0fWNDSScMv9ENOkhswuPwJ3iRa2wEJowV0UNk0sJQkFZr2lwqnfJr2aYJawcxg7sWVeP-lKPCA3i5TCrHaP1ospW7YpYbDMOMk


In [5]:
## Requête GET pour obtenir la data sur l'artiste
### Ici on utilise du python propre i.e requests.get() 

# We make a GET request and store the response 
# Add the Content-Type header set to the application/x-www-form-urlencoded value.
headers = {
    'Authorization': 'Bearer {}'.format(access_token)
}

# 2. Get artist's info: nb followers, popularity, genres

### a. First get the list of artists

In [7]:
filepath = os.path.join('..','data','most_streamed_spotify_ 2024_clean2.csv')
df = pd.read_csv(filepath)
df.head()

,Track,Album_Name,Artist,Release_Date,ISRC,All_Time_Rank,Track_Score,Spotify_Streams,Spotify_Playlist_Count,Spotify_Playlist_Reach,...,Deezer_Playlist_Count,Deezer_Playlist_Reach,Amazon_Playlist_Count,Pandora_Streams,Pandora_Track_Stations,Soundcloud_Streams,Shazam_Counts,TIDAL_Popularity,Explicit_Track,Pandora_Track
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,2024-04-26,QM24S2402528,1,725.4,390470936,30716,196631588,...,62,17598718,114,18004655,"22,931",4818457,2669262,NaN,0,18004655
1,Not Like Us,Not Like Us,Kendrick Lamar,2024-05-04,USUG12400910,2,545.9,323703884,28113,174597137,...,67,10422430,111,7780028,"28,444",6623075,1118279,NaN,1,7780028
2,i like the way you kiss me,I like the way you kiss me,Artemas,2024-03-19,QZJ842400387,3,538.4,601309283,54331,211607669,...,136,36321847,172,5022621,"5,639",7208651,5285340,NaN,0,5022621
3,Flowers,Flowers - Single,Miley Cyrus,2023-01-12,USSM12209777,4,444.9,2031280633,269802,136569078,...,264,24684248,210,190260277,"203,384",0,11822942,NaN,0,190260277
4,Houdini,Houdini,Eminem,2024-05-31,USUG12403398,5,423.3,107034922,7223,151469874,...,82,17660624,105,4493884,"7,006",207179,457017,NaN,1,4493884


In [12]:
df = df.sort_values('Spotify_Streams',ascending=False)
df.head()

,Track,Album_Name,Artist,Release_Date,ISRC,All_Time_Rank,Track_Score,Spotify_Streams,Spotify_Playlist_Count,Spotify_Playlist_Reach,...,Deezer_Playlist_Count,Deezer_Playlist_Reach,Amazon_Playlist_Count,Pandora_Streams,Pandora_Track_Stations,Soundcloud_Streams,Shazam_Counts,TIDAL_Popularity,Explicit_Track,Pandora_Track
55,Blinding Lights,Blinding Lights,The Weeknd,2019-11-29,USUG11904206,56,194.0,4281468720,590392,165544011,...,564,11701129,184,489299762,"319,634",28659471,28994660,NaN,0,489299762
4254,Blinding Lights,Blinding Lights,xSyborg,2021-03-13,QZLBC2302779,4229,20.5,4261328190,0,0,...,0,0,0,0,NaN,0,0,NaN,0,0
53,Shape of You,Shape of You,Ed Sheeran,2017-01-06,GBAHS1600463,54,195.1,3909458734,316378,148037882,...,512,2710931,115,733565655,"651,557",78149375,1146246,NaN,0,733565655
1637,Shape of You,Shape of You,xSyborg,2021-02-20,QZLBC2302789,1807,36.3,3888356417,0,0,...,0,0,0,0,NaN,0,0,NaN,0,0
132,Someone You Loved,Breach,Lewis Capaldi,2018-11-08,DEUM71807062,133,129.7,3427498835,338763,160553919,...,502,4834088,110,427212257,"357,540",0,29266297,NaN,0,427212257


### b. Now retrieve Spotify ID for those artists & get nb of followers 

In [23]:
def fetch_artist_id(row):
    artist_name= row['Artist']
    url_artist=artist_name.replace(' ','+')
    response_artist_search = requests.get(BASE_URL + 'search?q='+url_artist+'&type=artist', headers=headers) 
    
    if response_artist_search.status_code == 200: 
        artist_id = response_artist_search.json().get('artists')['items'][0]['id']
        return artist_id
    else:
        return "Error: "+ str(response_artist_search.status_code)


fetch_artist_id(df.iloc[0]) # checked and aligned with spotify app's info

'1Xyo4u8uXC1ZmMpatF05PJ'

In [24]:
df['spotify_artist_id']=df.apply(fetch_artist_id, axis=1)

KeyboardInterrupt: 

In [22]:
def fetch_followers(row):
    response_artist_found = requests.get(BASE_URL + 'artists/'+row['artist_id'], headers=headers) 

    if response_artist_found.status_code == 200: 
        followers = response_artist_found.json().get('followers')['total'] 
        return followers
    else:
        return "Error: "+ str(response_artist_found.status_code)

fetch_followers(df.iloc[0])

KeyError: 'artist_id'

In [ ]:
def fetch_genres(row):
    response_artist_found = requests.get(BASE_URL + 'artists/'+row['artist_id'], headers=headers) 
    if response_artist_found.status_code == 200: 
        genres = response_artist_found.json().get('genres')
        return genres
    else:
        return "Error: "+ str(response_artist_found.status_code)

fetch_genres(df.iloc[0])

In [ ]:
def fetch_artist_popularity(row):
    response_artist_found = requests.get(BASE_URL + 'artists/'+row['artist_id'], headers=headers) 
    if response_artist_found.status_code == 200: 
        genres = response_artist_found.json().get('popularity')
        return genres
    else:
        return "Error: "+ str(response_artist_found.status_code)


fetch_artist_popularity(df.iloc[0])

In [ ]:
df['spotify_followers_feb25']=df.apply(fetch_followers, axis=1)

In [ ]:
df['artist_genres']=df.apply(fetch_genres, axis=1)

In [ ]:
df['artist_popularity_feb25']=df.apply(fetch_artist_popularity, axis=1)

# 3. Get track info

### Search track id by inputting track name AND artist name (there might be several tracks with same title)

In [19]:
def fetch_markets(row):
    artist_name= row['Artist']
    url_artist=artist_name.replace(' ','+')
    url_track=row['Track'].replace(' ','+')

    response_track_search = requests.get(BASE_URL + 'search?q=track:'+url_track+'%20artist:'+url_artist+'&type=track&limit=10', headers=headers) 
    
    if response_track_search.status_code == 200: 
        track_id = response_track_search.json().get('tracks')['items'][0]['id']
        response_track_found = requests.get(BASE_URL + 'tracks/'+track_id, headers=headers) 
        if response_track_found.status_code == 200: 
            markets = response_track_found.json().get('available_markets')
            return markets
        else:
            return "Error: "+ str(response_track_found.status_code)
    else:
        return "Error: "+ str(response_track_search.status_code)


fetch_markets(df.iloc[0])

['AR',
 'AU',
 'AT',
 'BE',
 'BO',
 'BR',
 'BG',
 'CA',
 'CL',
 'CO',
 'CR',
 'CY',
 'CZ',
 'DK',
 'DO',
 'DE',
 'EC',
 'EE',
 'SV',
 'FI',
 'FR',
 'GR',
 'GT',
 'HN',
 'HK',
 'HU',
 'IS',
 'IE',
 'IT',
 'LV',
 'LT',
 'LU',
 'MY',
 'MT',
 'MX',
 'NL',
 'NZ',
 'NI',
 'NO',
 'PA',
 'PY',
 'PE',
 'PH',
 'PL',
 'PT',
 'SG',
 'SK',
 'ES',
 'SE',
 'CH',
 'TW',
 'TR',
 'UY',
 'US',
 'GB',
 'AD',
 'LI',
 'MC',
 'ID',
 'JP',
 'TH',
 'VN',
 'RO',
 'IL',
 'ZA',
 'SA',
 'AE',
 'BH',
 'QA',
 'OM',
 'KW',
 'EG',
 'MA',
 'DZ',
 'TN',
 'LB',
 'JO',
 'PS',
 'IN',
 'BY',
 'KZ',
 'MD',
 'UA',
 'AL',
 'BA',
 'HR',
 'ME',
 'MK',
 'RS',
 'SI',
 'KR',
 'BD',
 'PK',
 'LK',
 'GH',
 'KE',
 'NG',
 'TZ',
 'UG',
 'AG',
 'AM',
 'BS',
 'BB',
 'BZ',
 'BT',
 'BW',
 'BF',
 'CV',
 'CW',
 'DM',
 'FJ',
 'GM',
 'GE',
 'GD',
 'GW',
 'GY',
 'HT',
 'JM',
 'KI',
 'LS',
 'LR',
 'MW',
 'MV',
 'ML',
 'MH',
 'FM',
 'NA',
 'NR',
 'NE',
 'PW',
 'PG',
 'WS',
 'SM',
 'ST',
 'SN',
 'SC',
 'SL',
 'SB',
 'KN',
 'LC',
 'VC',
 'SR',
 'TL',

In [ ]:
df['track_markets']=df.apply(fetch_markets, axis=1)
df